In [21]:
"""
Evaluate the log likelihood of the data given the generated inhomogeneous rate function of the Poisson Process
"""
function log_likelihood(data::AbstractArray, θ::PoissonProcessParams, KernList::AbstractArray)
    C, α, u, N, M, D, J = name_params(θ)
    
    out = 0 # total log likelihood
    out1 = Array(Any,(N,M)); # log likelihood of individual [n,m] observations
    out2 = Array(Any,(N,M)) # Function values at spikes
    out3 = Array(Any,(N,M)) # Integrals of rate function at individual neurons
    for n = 1:N
        for m = 1:M
            # Get the predicted firing rates at the spikes
            frate = firing_rates(data[n][m], θ, KernList; n_range=n, m_range=m)[n,m]
            out2[n,m] = frate
            
            # Compute the numerical integral of the firing rates
            ∫frate_dt = hquadrature(t->(firing_rates(t, θ, KernList; n_range=n, m_range=m)[n,m][1]), 0.0, 1.0, reltol=1e-2, abstol=length(frate)*1e-2)[1]
            out3[n,m] = ∫frate_dt
            
            out1[n,m] = sum(log(frate)) - ∫frate_dt # ∑log(λ(t_i)) - ∫ λ(t) dt
            
            #assert(out1[n,m]<0)
        end
    end
    
    out = sum(out1[:])
    return out, out1, out2, out3
end

LoadError: LoadError: UndefVarError: PoissonProcessParams not defined
while loading In[21], in expression starting on line 433